# Expenses notebook

This is a notebook focused to understand where your money is being spent.

Below there are some basic configurations necessary to be used with this notebook. With them you can:

- Update table to convert some categories depending on data source
- Add your expense goals by category
- Add splitwise groups
- Define manual files to fetch

In [ ]:
NUBANK_CATEGORY_CONVERSION = {
    'supermercado': 'mercado',
    'eletrônicos':  'compras',
    'vestuário':    'compras',
    'ajuste':       'outros',
}


SPLITWISE_CATEGORY_CONVERSION = {
    'aluguel':                    'casa',
    'tv/telefone/internet':       'casa',
    'eletricidade':               'casa',
    'manutenção':                 'casa',
    'casa - outros':              'casa',
    'serviços':                   'casa',
    'móveis':                     'casa',
    'limpeza':                    'casa',
    'animais de estimação':       'casa',
    'aquecimento/gás':            'casa',
    'jantar fora':                'restaurante',
    'comidas e bebidas - outros': 'restaurante',
    'produtos de limpeza':        'mercado',
    'bebidas alcoólicas':         'mercado',
    'táxi':                       'transporte',
    'estacionamento':             'transporte',
    'transporte - outros':        'transporte',
    'filmes':                     'lazer',
    'música':                     'lazer',
    'bicicleta':                  'lazer',
    'jogos':                      'lazer',
    'entretenimento - outros':    'lazer',
    'esportes':                   'lazer',
    'despesas médicas':           'saúde',
    'eletrônicos':                'compras',
    'vestuário':                  'compras',
    'hotel':                      'viagem',
    'ônibus/trem':                'viagem',
    'avião':                      'viagem',
    'combustível':                'viagem',
    'carro':                      'viagem',
    'seguro':                     'outros',
    'vida - outros':              'outros',
    'geral':                      'outros',
    'presentes':                  'outros',
    'impostos':                   'outros',
    'creche':                     'outros',
}

EXPENSES_DISTRIBUTION = {
    'casa': 0.24,
    'restaurante': 0.09,
    'mercado': 0.09,
    'lazer': 0.02,
    'transporte': 0.02,
    'compras': 0.01,
    'saúde': 0.01
}

LOAD_CONFIGS = {
    'data_path': "./data", # Directory that contains the expenses data
    'nubank_file_pattern': "nubank*.csv",
    'splitwise_groups': [
        'jacas-keter',
        '2021-mozi-e-eu',
        '2020-mozi-e-eu',
        '2019-mozi-e-eu',
        '2018-mozi-e-eu'
    ],
    'manual_file_pattern': "Extrato outras contas*Despesas*.csv",
    'incomes_file_pattern': "Extrato outras contas*Rendimentos*.csv",
    'person_who_pays': "Lucas Alencar", # Person name that the expenses will be extracted from Splitwise csv
    'splitwise_category_table': SPLITWISE_CATEGORY_CONVERSION,
    'nubank_category_table': NUBANK_CATEGORY_CONVERSION,
}

In [ ]:
from datetime import date, timedelta
from date_helpers import previous_month

# Date used to make month analysis through the notebook
BASE_DATE = previous_month(date.today())

# Period used for analysis
BASE_PERIOD_START = BASE_DATE - timedelta(days=365)
BASE_PERIOD_END = BASE_DATE

# Year you were born
YEAR_OF_BIRTH = 1991

## Data load and preprocess
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import display
import record_summary as rs

from load import expenses
expenses = expenses.load(**LOAD_CONFIGS)

# Monthly view

In [ ]:
from load import filters
from expenses import monthly as monthly_expenses

MONTHLY_BALANCE_GOAL = 0.4
print(" >>> Monthly balance goal:", MONTHLY_BALANCE_GOAL * 100, "%")

# MONTHLY_EXPENSES_START_PERIOD = date(2018,1,1)
MONTHLY_EXPENSES_START_PERIOD = BASE_PERIOD_START
MONTHLY_EXPENSES_END_PERIOD = BASE_PERIOD_END

monthly_view = filters.by_monthly_period(expenses, 
                                         MONTHLY_EXPENSES_START_PERIOD, 
                                         MONTHLY_EXPENSES_END_PERIOD)

expenses_period = monthly_view[monthly_view.category != 'renda']
incomes_period = monthly_view[monthly_view.category == 'renda']

monthly_exp = monthly_expenses.month_by_month_summary(expenses_period, incomes_period)
display(monthly_expenses.style_summary(monthly_exp, MONTHLY_BALANCE_GOAL))

avg_monthly_exp = monthly_expenses.avg_month_summary(expenses_period, incomes_period)
display(monthly_expenses.style_summary(avg_monthly_exp, MONTHLY_BALANCE_GOAL))

monthly_expenses.plot(monthly_exp)

# Category view

In [ ]:
from expenses import category

print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

month_view = rs.records_for_month(expenses, BASE_DATE)

expenses_for_month = month_view[month_view.category != 'renda']
incomes_for_month = month_view[month_view.category == 'renda']

expenses_for_category_by_total = category.distribution(expenses_for_month, incomes_for_month, EXPENSES_DISTRIBUTION)
display(category.style_distribution(expenses_for_category_by_total))

from plotting import pie
pie.plot(expenses_for_category_by_total['% by expenses'], height=400)

# TODO Pie plot for % by income

In [ ]:
import pandas as pd

def list_category_expenses(category):
    print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)
    max_rows =  pd.get_option('display.max_rows')
    pd.set_option('display.max_rows', None) 

    # BASE_DATE = date(2020, 7, 1)
    month_view = rs.records_for_month(expenses, BASE_DATE)
    
    print("Expenses for category", category)
    display(month_view[(month_view.category == category)].sort_values(['date', 'title']))
    pd.set_option('display.max_rows', max_rows) 

def top_expenses(category, top):
    print("TOP", top, "expenses for", category)
    display(month_view[(month_view.category == category)].nsmallest(top, 'amount'))

In [ ]:
list_category_expenses('casa')
top_expenses('casa', 10)

In [ ]:
list_category_expenses('restaurante')
top_expenses('restaurante', 10)

In [ ]:
list_category_expenses('mercado')

In [ ]:
list_category_expenses('educação')

In [ ]:
list_category_expenses('saúde')

In [ ]:
list_category_expenses('lazer')

In [ ]:
list_category_expenses('serviços')

In [ ]:
list_category_expenses('transporte')

In [ ]:
list_category_expenses('compras')

In [ ]:
list_category_expenses('viagem')

In [ ]:
list_category_expenses('outros')
top_expenses('outros', 10)

In [ ]:
from load import filters
from expenses import category

# CATEGORY_EXPENSES_START_PERIOD = date(2018,1,1)
CATEGORY_EXPENSES_START_PERIOD = BASE_PERIOD_START
CATEGORY_EXPENSES_END_PERIOD = BASE_PERIOD_END

monthly_view = filters.by_monthly_period(expenses, 
                                         CATEGORY_EXPENSES_START_PERIOD, 
                                         CATEGORY_EXPENSES_END_PERIOD)

# Plots over time for each category grouping them by easy of read
expenses_over_time_data = category.add_food_expenses(monthly_view, 
                                                     category.over_time(monthly_view, expenses, 'amount #'))

from plotting import over_time
over_time.plot(expenses_over_time_data[['casa']])
over_time.plot(expenses_over_time_data[['restaurante', 'mercado', 'alimentação']])
over_time.plot(expenses_over_time_data[['transporte', 'lazer', 'serviços', 'saúde']])
over_time.plot(expenses_over_time_data[['viagem', 'educação', 'compras']])

# How much my time is worth?

In [ ]:
from incomes import salary

HOURS_WORKED_BY_DAY = 8.8 # According to CLT

# BASE_DATE = date(2019,7,1)

earned_for_month = salary.earned_for_month(expenses, BASE_DATE)
hours_worked_by_month = salary.hours_worked_for_month(HOURS_WORKED_BY_DAY, BASE_DATE)
money_for_time = earned_for_month / hours_worked_by_month

msg = """In {}, you have earned {}.
Given you have worked for {:,.2f} hours in this month.
Your time is worth {:,.2f} R$/hour.
Not considering holidays or vacations."""

print(msg.format(BASE_DATE.strftime('%Y-%m'), earned_for_month, hours_worked_by_month, money_for_time))
print("Work {:,.2f} hours to get R$ 100".format(100 / money_for_time))